# Inventory Management with Object Detection and SingleStore Database

This notebook explores an advanced inventory management system leveraging state-of-the-art object detection techniques with the YOLO (You Only Look Once) model and integrating OpenAI's GPT-4 for vision-based object detection insights. The system is designed to automate inventory tracking, offering a sophisticated approach to recognizing, counting, and managing stock levels through visual analysis.

## Objectives

- **Model Training**: Utilize YOLO for training a custom object detection model tailored to identify specific inventory items within images.
- **Object Detection**: Implement object detection on inventory images to locate and identify items using the trained YOLO model.
- **Data Management**: Store and manage detected bounding box information using SingleStore, a high-performance, SQL-compliant database optimized for real-time analytics.
- **Insight Generation**: Apply OpenAI's GPT-4 vision capabilities to interpret images and provide actionable inventory insights.

## Technologies Used

- **YOLO (You Only Look Once)**: A fast, accurate object detection framework, ideal for real-time applications.
- **SingleStore Database**: A distributed SQL database that excels in fast data ingestion and real-time analytics.
- **OpenAI GPT-4**: Leveraging the latest in AI to provide vision-based analysis and natural language processing.

By the end of this notebook, you will have a comprehensive understanding of how to build and deploy a cutting-edge inventory management system that bridges the gap between traditional stock-keeping methods and AI-driven automation.


Check the GPU status and configuration with `nvidia-smi`, ensuring that the environment is ready for deep learning tasks.

In [ ]:
!nvidia-smi

## Environment Setup and Library Import

In this section, we prepare our working environment for object detection tasks:

1. **Determine the Current Working Directory**: Establish a reference to our working directory, which is crucial for relative path operations and file management throughout the notebook.
2. **Install the Ultralytics Package**: We install a specific version of the `ultralytics` library, which provides an easy-to-use interface for working with YOLO models. This ensures compatibility and access to the latest features necessary for our object detection tasks.
3. **Clear Output**: The output of the installation process is cleared for a cleaner notebook presentation.
4. **Library Imports and Checks**: We import necessary libraries, including `ultralytics` for object detection and IPython utilities for better output management. Additionally, we perform system checks to ensure our environment is correctly set up for utilizing YOLO models with the `ultralytics.checks()` function.

This initial setup is essential for ensuring that all subsequent operations, from model training to object detection, run smoothly in a well-prepared environment.


In [ ]:
import os
HOME = os.getcwd()
print(HOME)

In [ ]:
!pip install ultralytics==8.0.196

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

In [5]:
from ultralytics import YOLO
from IPython.display import display, Image

## Training the YOLO Model

In this crucial step, we commence the training of our YOLO object detection model with the following parameters:

- **Task**: We specify `detect` as our primary task, focusing on detecting objects within images.
- **Mode**: Set to `train`, indicating that we are in the training phase of our model development.
- **Model**: Utilizes `yolov8s.pt`, a pre-trained YOLOv8 small model, serving as our starting point for training. This choice balances efficiency and accuracy.
- **Data**: Points to our dataset configuration file `data.yaml` located within the `supermarket_dataset` directory. This file contains all necessary information about our training and validation datasets.
- **Epochs**: The model will go through 25 training epochs, allowing it to learn from the dataset iteratively.
- **Image Size**: Sets the image resolution to `640x640` pixels, which is a common size for balancing model performance and detection accuracy.
- **Plots**: Enabled to generate and display training progress plots, offering insights into how the model's performance evolves over time.

This training phase is pivotal, as it fine-tunes the pre-trained YOLO model on our specific supermarket dataset, enabling it to recognize and locate items from the inventory effectively.


In [ ]:
!yolo task=detect mode=train model=yolov8s.pt data={"/content/drive/MyDrive/supermarket_dataset"}/data.yaml epochs=25 imgsz=640 plots=True # use your own dataset path


Load the trained YOLO model and run detection on an image to obtain results, specifying a confidence threshold of 0.5.

In [ ]:
model = YOLO(f'/content/runs/detect/train/weights/best.pt')
results = model(source='/content/cleaning-store.jpg', conf=0.5)


After loading the YOLO model with custom-trained weights, we perform object detection on a specific image, adjusting the confidence threshold to 0.25. The detected objects are then visualized and saved as an image, showcasing the model's ability to identify items within the scene.

In [ ]:
from ultralytics import YOLO
from PIL import Image

# Load the model
model = YOLO('/content/runs/detect/train/weights/best.pt')  # Adjust path as necessary

# Perform detection
results = model(source='/content/coke.jpg', conf=0.25)

# Show the results
for r in results:
    im_array = r.plot()  # plot a BGR numpy array of predictions
    im = Image.fromarray(im_array[..., ::-1])  # RGB PIL image
    im.show()  # show image
    im.save('results.jpg')  # save image

Utilizing the YOLO model, object detection is executed on an image with a specified confidence threshold of 0.25, focusing on accurately identifying objects. The output includes the bounding boxes for each detected object, which are extracted and converted into a list format for further analysis. This process effectively demonstrates how the model discerns and locates items within an image, highlighting the precise coordinates of each object's boundaries.

In [ ]:
from ultralytics import YOLO
import torch


model = YOLO('/content/runs/detect/train/weights/best.pt')  # Adjust path as necessary

# Perform detection
results = model(source='/content/coke.jpg', conf=0.25)

boxes = []
# View results
for r in results:
    boxes.append(r.boxes.xyxy.tolist())


print(boxes)

## Integration with SingleStore for Bounding Box Data Management

This section outlines the steps for integrating object detection results with SingleStore, a highly scalable SQL database, to store and manage bounding box data efficiently. The process involves several key steps, from setting up the database connection to creating a dedicated table for the bounding boxes and inserting detection results.

### Installing SingleStoreDB Library

First, we install the `singlestoredb` Python library, which facilitates interaction with SingleStore databases directly from Python applications. This library allows us to execute SQL commands and manage data within SingleStore seamlessly.


In [ ]:
!pip install singlestoredb


### Establishing Database Connection

A connection to the SingleStore database is established using the `singlestoredb.connect` method, which requires specifying connection parameters such as the username, password, and database URL. This connection enables us to execute SQL commands and interact with the database:


In [50]:
import singlestoredb as s2
conn_params = 'user:password@host:port/database' #enter your database params
conn = s2.connect(conn_params)


### Creating the BoundingBoxes Table

We define and execute a SQL command to create a `BoundingBoxes` table if it does not already exist. This table is designed to store the bounding box coordinates (`X1`, `Y1`, `X2`, `Y2`) along with an associated `ImageID`, which links each bounding box to a specific image in our dataset:


In [ ]:
# SQL command to create the BoundingBoxes table
create_table_command = """
CREATE TABLE IF NOT EXISTS BoundingBoxes (
    BoxID INT AUTO_INCREMENT PRIMARY KEY,
    ImageID INT,
    X1 FLOAT,
    Y1 FLOAT,
    X2 FLOAT,
    Y2 FLOAT
);
"""



# Execute the SQL command
with s2.connect(conn_params) as conn:
    with conn.cursor() as cursor:
        cursor.execute(create_table_command)
        print("Table 'BoundingBoxes' created successfully.")

### Inserting Detection Results

With the `BoundingBoxes` table in place, we prepare an SQL statement to insert bounding box data for each detected object. This step involves iterating over the list of detected bounding boxes obtained from the YOLO model and inserting each box's coordinates into the table. By associating each bounding box with an `ImageID`, we create a structured and queryable record of detections:

In [37]:
# Prepare the SQL statement for inserting bounding box data
stmt = 'INSERT INTO BoundingBoxes (ImageID, X1, Y1, X2, Y2) VALUES (%s, %s, %s, %s, %s)'

# Assuming each bounding box is associated with the same image, for example, ImageID = 1
image_id = 1  # Adjust this based on your actual ImageID

with s2.connect(conn_params) as conn:
    conn.autocommit(True)
    with conn.cursor() as cur:
        # Iterate over each bounding box and insert its data
        for box in boxes[0]:  # Assuming boxes[0] contains your bounding box data for one image
            cur.execute(stmt, (image_id, box[0], box[1], box[2], box[3]))


This integration not only facilitates the efficient storage and management of detection results but also lays the groundwork for further analysis and querying of object detection data within the SingleStore database. By leveraging SingleStore's capabilities, we can perform real-time analytics and gain deeper insights into the detected objects across our image dataset.


## Leveraging GPT-4 for Vision-Based Inventory Insights

In this innovative approach, we utilize OpenAI's GPT-4 with vision capabilities to analyze inventory levels directly from images. By integrating GPT-4's advanced understanding of visual content, we aim to extract quantitative insights that inform inventory management decisions.

### Implementation Details

We employ the `openai` Python library to interface with GPT-4's API, specifically its vision model variant. The process involves sending an image URL to the model along with a prompt that instructs GPT-4 to interpret the image in the context of inventory management. The model is tasked with estimating the percentage of inventory remaining, providing a concise and actionable piece of information:


In [ ]:
!pip install openai

In [ ]:
from openai import OpenAI
from google.colab import userdata

client = OpenAI(api_key=userdata.get('openai_key')) # store your openai key secretly in env variables

response = client.chat.completions.create(
  model="gpt-4-vision-preview",
  messages=[
      {"role": "system", "content": "You are an inventory management bot. Your only goal is to output a percentage. This percentage is the amount of inventory left based on the input image."},
    {
      "role": "user",
      "content": [
        {
          "type": "image_url",
          "image_url": {
            "url": "https://www.redmetal.eu/wp-content/uploads/2021/08/Warehouse-Pallet-Racking-Manufacturer-07.webp",
          },
        },
      ],
    }
  ],
  max_tokens=300,
)

print(response.choices[0].message)

## Considerations for AI-Driven Inventory Analysis

- **Explainability**: Understanding the reasoning behind the AI's percentage estimation is challenging. Multimodal LLMs, especially those as complex as GPT-4, are often considered "black boxes" due to their opaque decision-making processes. This can make it difficult to fully trust or understand the basis of their predictions.

- **Reliability**: While powerful, AI predictions can sometimes be unreliable or inconsistent, particularly in nuanced or ambiguous contexts. It's essential to validate these automated insights with real-world data and adjust strategies accordingly.

- **Future Improvements**: As AI technology and vision models continue to evolve, we can expect improvements in accuracy, reliability, and explainability. These advancements will further enhance the utility of AI in inventory management and beyond.

By integrating GPT-4's vision capabilities into our inventory management toolkit, we unlock new possibilities for automated, image-based analysis. However, it's crucial to approach these AI-driven insights with a critical eye, acknowledging the current limitations while looking forward to future advancements.
